In [1]:
import numpy as np
import pandas as pd

In [1367]:
book_map = pd.read_csv('books.csv')

C:\Users\snoza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [1368]:
book_map.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg


In [352]:
df = pd.read_csv('ratings.csv')

In [353]:
df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [355]:
df['ISBN'] = df['ISBN'].str.replace(')','').replace('(','').replace(',','').replace('ISBN','')

In [356]:
#總用戶數
users = df["User-ID"].unique().tolist()
len(users)

105283

In [489]:
#總評分數
ratings = df[df['Book-Rating']>=5]["User-ID"].unique().tolist()
len(ratings)

75117

In [490]:
#總書本數
books = df[df['Book-Rating']>=5]["ISBN"].unique().tolist()
len(books)

179376

In [1028]:
df_ = df[(df['Book-Rating']>=5) | (df['Book-Rating']==0)]

In [1065]:
#排除評論次數不到100次的書
df_revise = df_.groupby('ISBN').filter(lambda x: len(x) > 100)

In [1066]:
#更新後總書本數
books_revise = df_revise["ISBN"].unique().tolist()
len(books_revise)

702

In [1067]:
#排除評論次數不到2次的用戶(為取1筆資料做test_set)
df_revise2 = df_revise.groupby('User-ID').filter(lambda x: len(x) >= 2)

In [1068]:
#評論2本以上不同書用戶數
users_revise = df_revise2["User-ID"].unique().tolist()
len(users_revise)

12867

In [1369]:
#更新書名字典
book_map = pd.merge(pd.DataFrame(df_revise2['ISBN'].unique(), columns = ['ISBN']), book_map)

In [1069]:
#切分train/test set
df_test_value = df_revise2.groupby(['User-ID']).tail(1)

In [1070]:
df_test_index = df_test_value.index.to_list()

In [1071]:
df_test = df_revise2[df_revise2.index.isin(df_test_index)]

In [1072]:
df_train = df_revise2[~df_revise2.index.isin(df_test_index)]

In [1073]:
df_train

,User-ID,ISBN,Book-Rating
10,276746,0425115801,0
11,276746,0449006522,0
12,276746,0553561618,0
133,276822,0060096195,10
164,276832,0316781266,0
...,...,...,...
1149712,276688,0553569155,0
1149726,276688,0679751521,0
1149761,276704,0345386108,6
1149768,276704,0446605409,0


In [1074]:
df_test

,User-ID,ISBN,Book-Rating
13,276746,055356451X,0
145,276822,0786817070,10
165,276832,0345446860,5
457,276925,0971880107,0
625,276939,0971880107,0
...,...,...,...
1149432,276670,1573227331,7
1149592,276680,0743203631,7
1149646,276681,0399144463,8
1149731,276688,068484267X,0


In [1075]:
def to_book_df(df):
    df['User-ID'] = df['User-ID'].astype('str')
    df['ISBN'] = df['ISBN'].astype('str')
    cust_book = df.groupby(['User-ID'])
    cust_book_dict = {}
    for gp in cust_book.groups:
        cust_book_list = {gp:cust_book.get_group(gp)['ISBN'].tolist()}
        cust_book_dict.update(cust_book_list)
    
    df_cust_book = pd.DataFrame(list(cust_book_dict.items()), columns = ['gp', 'book_list'])
        
    return df_cust_book

In [1076]:
df_gp_train = to_book_df(df_train)

<ipython-input-1075-2721b2ff06f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User-ID'] = df['User-ID'].astype('str')
<ipython-input-1075-2721b2ff06f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ISBN'] = df['ISBN'].astype('str')


In [1077]:
df_gp_train

,gp,book_list
0,100004,"[0061015725, 0345339703, 0345339711, 034533973..."
1,100009,"[0060392452, 0060502258, 0312982518, 034539105..."
2,10001,"[0316601950, 0425092917, 0446359866, 0451160525]"
3,100035,"[0440211727, 0553277472, 0671673688]"
4,100053,"[0312195516, 0312422156, 0316769487, 0375703055]"
...,...,...
12862,99955,"[0316666343, 0345313860, 0375760911, 038572010..."
12863,99963,"[0345370775, 0449223604]"
12864,99980,[0140119906]
12865,99996,"[0440235162, 044023722X, 0440241073]"


In [1078]:
df_gp_test = to_book_df(df_test)

<ipython-input-1075-2721b2ff06f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User-ID'] = df['User-ID'].astype('str')
<ipython-input-1075-2721b2ff06f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ISBN'] = df['ISBN'].astype('str')


In [1079]:
import gensim
from gensim.models import Word2Vec

In [1080]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [1081]:
book_lists = list(df_gp_train['book_list'])

In [1082]:
import random

for book_list in book_lists:
    random.shuffle(book_list)

In [1211]:
import datetime
start = datetime.datetime.now()

model = Word2Vec(sentences = book_lists,
                vector_size = 50, 
                min_count = 1,                
                workers = cores-1,
                sg = 1,
                hs = 0,
                negative = 30,
                window = 999,
                epochs = 20)

print("Time passed:"+str(datetime.datetime.now()-start))
model.save('item2vec_v1')

Time passed:0:02:09.428180


In [1212]:
word_vectors = model.wv

In [1213]:
def recommender(positive_list = None, topn = 10):
    recommend_prod = []
    for book_id, prob in model.wv.most_similar(positive = positive_list, topn = topn):
        recommend_prod.append(book_id)
    return recommend_prod

In [1214]:
recommender(positive_list = ['0345339703'], topn = 5)

['0345339681', '0345339738', '0345339711', '0441790348', '0451184963']

In [1245]:
def cust_recommend_smy(df):
    recommend_smy = []
    for index, row in df.iterrows():
        cust_id = row['gp']
        book_list = row['book_list']
        book_list = list(book_list)
        prod_recommend_list = recommender(positive_list = book_list, topn = 30)
        recommend_smy.append([cust_id, prod_recommend_list])
                             
    df_recommend = pd.DataFrame(recommend_smy, columns = ['cust_id', 'recommend_list'])
    
    return df_recommend

In [1246]:
start = datetime.datetime.now()
df_recommend = cust_recommend_smy(df_gp_train)

print("Time passed:"+str(datetime.datetime.now()-start))  

Time passed:0:00:05.955097


In [1247]:
df_recommend.to_csv('df_recommend.csv')

In [1248]:
predict_smy = pd.merge(df_recommend, df_gp_test, left_on='cust_id', right_on='gp')

In [1249]:
predict_smy

,cust_id,recommend_list,gp,book_list
0,100004,"[0345339681, 0345370775, 0345342968, 034540288...",100004,[059035342X]
1,100009,"[044023722X, 0440206154, 0440221471, 044022060...",100009,[0553582747]
2,10001,"[0553268880, 0440211727, 0440220602, 044022467...",10001,[059035342X]
3,100035,"[044021145X, 0345370775, 0380018179, 044022467...",100035,[0671888587]
4,100053,"[0316666343, 0679781587, 0971880107, 006092833...",100053,[0679746048]
...,...,...,...,...
12862,99955,"[0060928336, 0553268880, 0440226430, 031660195...",99955,[0971880107]
12863,99963,"[0553280341, 0449221504, 0553279912, 044922151...",99963,[0786881852]
12864,99980,"[0385484518, 0312195516, 0060976845, 006093053...",99980,[0679746048]
12865,99996,"[0440236673, 0440225701, 0440221471, 044023474...",99996,[0452282152]


In [1250]:
def accuracy_rate(df):
    #df['recommend_list'] = df['recommend_list'].astype('str')
    #df['book_list'] = df['book_list'].astype('str')
    count = 0
    correct_count = 0
    for index, row in df.iterrows():
        ground_truth = str(row['book_list']).replace('[','').replace(']','').replace("'",'')
        recommend_list = row['recommend_list']
        if ground_truth in recommend_list:
            correct_count += 1
            count += 1
        else:count +=1
    
    accuracy = correct_count / count
    
    return print("correct_count is " + str(correct_count) + " count is " + str(count) + " accuracy_rate is " + str(accuracy))

In [1251]:
accuracy_rate(predict_smy)

correct_count is 2817 count is 12867 accuracy_rate is 0.21893215201678712


In [1389]:
def recommender_system(book_name):

    book_id = book_map[book_map['Book-Title'] == book_name]['ISBN']
    book_id = list(book_id)

    recommender_list = recommender(positive_list = book_id, topn = 30)
    recommender_book = []
    for book_id in recommender_list:
        book_name = book_map[book_map['ISBN'] == book_id]['Book-Title'].to_string(index = False)

        recommender_book.append(book_name)
    return recommender_book

In [1390]:
book_name = "The Fellowship of the Ring (The Lord of the Rings, Part 1)"
recommender_system(book_name)

[' The Hobbit : The Enchanting Prelude to The Lord of the Rings',
 ' The Return of the King (The Lord of the Rings, Part 3)',
 ' The Two Towers (The Lord of the Rings, Part 2)',
 ' Stranger in a Strange Land (Remembering Tomorrow)',
 ' Insomnia',
 ' Needful Things',
 ' The Giver (21st Century Reference)',
 " Tom Clancy's Op-Center: Mirror Image (Tom Clancy's Op Center (Paperback))",
 ' Silence of the Lambs',
 ' The Tommyknockers',
 ' The Client',
 ' Jurassic Park',
 " Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))",
 ' Fahrenheit 451',
 ' A Wrinkle in Time',
 ' The Catcher in the Rye',
 ' Lord of the Flies',
 ' Anne of Green Gables (Anne of Green Gables Novels (Paperback))',
 ' The Perfect Storm : A True Story of Men Against the Sea',
 ' The Green Mile',
 ' Watership Down',
 ' The Bourne Identity',
 ' The Loop',
 ' From the Corner of His Eye',
 ' The Golden Compass (His Dark Materials, Book 1)',
 " Don't Stand Too Close to a Naked Man",
 ' Politically Correct Bedtime Stori

In [1313]:
pd.set_option('max_colwidth', 100)